In [67]:
import numpy as np
import seaborn as sns
from feature_extractor import FeatureExtractor
from PIL import Image
import time
from pathlib import Path 
import os 
import matplotlib.pyplot as plt

In [68]:
fe = FeatureExtractor() 

Using cache found in C:\Users\juanm/.cache\torch\hub\pytorch_vision_v0.10.0


## Loading time

In [69]:
scale = [1, 10, 100, 1000, 10000]
times = []
paths = [x for x in Path(os.path.join('static', 'feature')).glob('*.npy')]

for s in scale:
    start_time = time.time()
    features = [] 
    img_paths = []
    for i in range(s):
        for feature_path in paths:
            features.append(np.load(feature_path)) 
            img_paths.append(Path("./static/img") / (feature_path.stem + ".jpg"))
    features = np.array(features)
    
    times.append(time.time() - start_time)
    features = features.squeeze(axis=1)

#### for ~20k images it is expected a loading time of: ~ 10 sec.

In [72]:
print((times[-1]/(8*scale[-1]))*20000, 'secs')

10.629500389099121 secs


## query time

In [73]:
scale = [1, 10, 100, 1000, 10000]
qtimes = []
paths = [x for x in Path(os.path.join('static', 'feature')).glob('*.npy')]

for s in scale:
    features = [] 
    img_paths = []
    for i in range(s):
        for feature_path in paths:
            features.append(np.load(feature_path)) 
            img_paths.append(Path("./static/img") / (feature_path.stem + ".jpg"))
    features = np.array(features)
    
    start_time = time.time()
    request_path = os.path.join('static','img', 'a1.jpg')
    img=Image.open(request_path).convert('RGB')
    query = fe.extract(img)
    query = query.numpy()
    query = np.expand_dims(query, axis=0)

    dists = np.linalg.norm(features-query, axis=2)
    dists = np.squeeze(dists, axis=1)

    ids = np.argsort(dists, axis=0)
    scores = [(dists[id], img_paths[id]) for id in ids]

    qtimes.append(time.time() - start_time)

#### for ~20k images it is expected a query time of: ~ 0.2 secs

In [79]:
print((qtimes[-1]/(8*scale[-1]))*20000, 'secs')

0.23600751161575317 secs
